In [1]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import altair as alt
from requests.utils import quote
import os
from github import Github
from datetime import timedelta

In [2]:
fmt = "{:%Y-%m-%d}"

# Can optionally use number of days to choose dates
n_days = 60
end_date = fmt.format(pd.datetime.today())
start_date = fmt.format(pd.datetime.today() - timedelta(days=n_days))

renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [3]:
# Parameters
end_date = "2019-07-16"
start_date = "2019-05-17"
renderer = "kaggle"


In [4]:
# Calculate number of days to include in plots
n_days = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days

In [5]:
# Create the renderer we want
alt.renderers.enable(renderer);

In [6]:
# Define our query object that we'll re-use for github search
class GitHubGraphQlQuery():
    def __init__(self, query):
        self.query = query
        self.headers = {"Authorization": "Bearer %s" % os.environ['GITHUB_ACCESS_TOKEN']}
        self.gql_template = """
{
  search(%s) {
    issueCount
    pageInfo {
        endCursor
        hasNextPage
    }
    nodes {
      ... on PullRequest {
        state
        id
        title
        url
        createdAt
        updatedAt
        number
        authorAssociation
        author {
          login
          url
        }
        mergedBy {
          login
          url
        }
        comments(last: 15) {
          edges {
            node {
              authorAssociation
              createdAt
              lastEditedAt
              url
              repository {
                id
              }
              author {
                login
                url
              }
            }
          }
        }
      }
      ... on Issue {
        url
        title
        state
        createdAt
        updatedAt
        number
        authorAssociation
        author {
          login
          url
        }
        comments(last: 15) {
          edges {
            node {
              author {
                login
                url
              }
              authorAssociation
              createdAt
              lastEditedAt
              repository {
                id
              }
              url
            }
          }
        }
      }
    }
  }
}
"""
    def request(self, n_pages=100, n_per_page=50):
        self.raw_data = []
        for ii in range(n_pages):
            search_query = ["first: %s" % n_per_page, 'query: "%s"' % self.query, 'type: ISSUE']
            if ii != 0:
                search_query.append('after: "%s"' % pageInfo['endCursor'])

            this_query = self.gql_template % ', '.join(search_query)
            request = requests.post('https://api.github.com/graphql', json={'query': this_query}, headers=self.headers)
            if request.status_code != 200:
                raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, this_query))
            if "errors" in request.json().keys():
                raise Exception("Query failed to run with error {}. {}".format(request.json()['errors'], this_query))
            self.request = request

            # Parse the response
            json = request.json()['data']['search']
            if ii == 0:
                print("Found {} items, which will take {} pages".format(json['issueCount'], int(np.ceil(json['issueCount'] / n_per_page))))
            self.raw_data.append(json)
            pageInfo = json['pageInfo']
            self.last_query = this_query
            if pageInfo['hasNextPage'] is False:
                break
        
        if self.raw_data[0]['issueCount'] == 0:
            print("Found no entries for query {}".format(self.query))
            self.data = None
            return
        
        # Add some extra fields
        self.data = pd.DataFrame([jj for ii in self.raw_data for jj in ii['nodes']])
        self.data['username'] = self.data['author'].map(lambda a: a['login'])
        self.data['username_url'] = self.data['author'].map(lambda a: a['url'])
        self.data['org'] = self.data['url'].map(lambda a: a.split('/')[3])
        self.data['repo'] = self.data['url'].map(lambda a: a.split('/')[4])

In [7]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [8]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [9]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [10]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [11]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (98 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (50 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (15 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (14 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [Jupyter community workshop: proposal on diversity, inclusion and accessibility](https://discourse.jupyter.org/t/jupyter-community-workshop-proposal-on-diversity-inclusion-and-accessibility) (20 posts)
* [Sunset the GitHub repo jupyter/help?](https://discourse.jupyter.org/t/sunset-the-github-repo-jupyter-help) (17 posts)
* [Reviving the all-Jupyter community meetings](https://discourse.jupyter.org/t/reviving-the-all-jupyter-community-meetings) (17 posts)

## Active topics on Discourse

These are topics with lots of activity in recent history.

In [12]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (98 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (51 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (50 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (31 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (28 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [Rename this category to Jupyter Server?](https://discourse.jupyter.org/t/rename-this-category-to-jupyter-server) (21 posts)
* [Potential collaboration on user research?](https://discourse.jupyter.org/t/potential-collaboration-on-user-research) (21 posts)

In [13]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:            category_id  bookmarked     parent_category  \
  10       accessibility           2      Special Topics   
  9       Admin requests           1       Site Feedback   
  0                 Meta           5                Meta   
  1        announcements           1                Meta   
  2           JupyterHub           2          JupyterHub   
  6              discuss           2          JupyterHub   
  8   Enterprise Gateway           1  Enterprise Gateway   
  3       Community Chat           3      Community Chat   
  4               Binder           6              Binder   
  7              discuss           6              Binder   
  5            BinderHub           1              Binder   
  
                            parent/category  
  10           Special Topics/accessibility  
  9            Site Feedback/Admin requests  
  0                               Meta/Meta  
  1                      Meta/announcements  
  2                   JupyterHub/JupyterHub  
  6                      JupyterHub/discuss  
  8   Enterprise Gateway/Enterprise Gateway  
  3           Community Chat/Community Chat  
  4                           Binder/Binder  
  7                          Binder/discuss  
  5                        Binder/BinderHub  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['accessibility', 'Admin requests', 'Meta', 'announcements', 'JupyterHub', 'discuss', 'Enterprise Gateway', 'Community Chat', 'Binder', 'discuss', 'BinderHub']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [14]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [Why does nb2kg connect to the kernel via EG rather than directly?](https://discourse.jupyter.org/t/why-does-nb2kg-connect-to-the-kernel-via-eg-rather-than-directly) (12 posts)
* [Boilerplate needed for making a Jupyterlab extension using Vue.js](https://discourse.jupyter.org/t/boilerplate-needed-for-making-a-jupyterlab-extension-using-vue-js) (6 posts)
* [Minimal layout for newbies](https://discourse.jupyter.org/t/minimal-layout-for-newbies) (5 posts)
* [Recent activity across all of Jupyter: a notebook report](https://discourse.jupyter.org/t/recent-activity-across-all-of-jupyter-a-notebook-report) (5 posts)
* [Technology choices for new Jupyter.org website](https://discourse.jupyter.org/t/technology-choices-for-new-jupyter-org-website) (4 posts)
* [Can JupyterLab access/use user installed fonts?](https://discourse.jupyter.org/t/can-jupyterlab-access-use-user-installed-fonts) (4 posts)
* [Add entries to the filebrowser contextmenu](https://discourse.jupyter.org/t/add-entries-to-the-filebrowser-contextmenu) (4 posts)
* [What is with the weird jovyan user?](https://discourse.jupyter.org/t/what-is-with-the-weird-jovyan-user) (3 posts)
* [Configure initial layout](https://discourse.jupyter.org/t/configure-initial-layout) (3 posts)
* [What is the best way to add numbers to equations in a Markdown cell in a Jupyter Lab notebook?](https://discourse.jupyter.org/t/what-is-the-best-way-to-add-numbers-to-equations-in-a-markdown-cell-in-a-jupyter-lab-notebook) (2 posts)

In [15]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:           category_id  bookmarked     parent_category  \
  6                 Q&A           0                 Q&A   
  9            Notebook           2            Notebook   
  8           nbconvert           1            Notebook   
  0                Meta           2                Meta   
  5          JupyterLab           5          JupyterLab   
  1          JupyterHub           0          JupyterHub   
  7  Enterprise Gateway           1  Enterprise Gateway   
  2      Community Chat           1      Community Chat   
  3              Binder           0              Binder   
  4             discuss           0              Binder   
  
                           parent/category  
  6                                Q&A/Q&A  
  9                      Notebook/Notebook  
  8                     Notebook/nbconvert  
  0                              Meta/Meta  
  5                  JupyterLab/JupyterLab  
  1                  JupyterHub/JupyterHub  
  7  Enterprise Gateway/Enterprise Gateway  
  2          Community Chat/Community Chat  
  3                          Binder/Binder  
  4                         Binder/discuss  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['Q&A', 'Notebook', 'nbconvert', 'Meta', 'JupyterLab', 'JupyterHub', 'Enterprise Gateway', 'Community Chat', 'Binder', 'discuss']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## User activity in Discourse

These are the top posters and top "like-ers" in the Discourse

In [16]:
def plot_user_data(users, column, sort=False):
    plt_data = users.sort_values(column, ascending=False).head(50)
    x = alt.X("username", sort=plt_data['username'].tolist()) if sort is True else 'username'
    ch = alt.Chart(data=plt_data).mark_bar().encode(
        x=x,
        y=column
    )
    return ch

In [17]:
url = "https://discourse.jupyter.org/directory_items.json"
params = {"period": "quarterly", "order": "post_count"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
users = pd.DataFrame(resp.json()['directory_items'])
users['username'] = users['user'].map(lambda a: a['username'])

In [18]:
plot_user_data(users.head(50), 'post_count')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             78    6          216             127         160        1374   
  1             90    3          197             116         135         988   
  2             65   24           16              51          71         668   
  3             82    8           10              59          53         830   
  4             64   12            9              48          47         844   
  5             71  116           21              51          44         610   
  6             77   73           17              30          35        1527   
  7             15  220            1              24          27         189   
  8             62  239            5               7          25         438   
  9             81   30           77              17          24         324   
  10            58   99           43              19          22         533   
  11            30  485           42              17          18         246   
  12            19  234            5               3          16          84   
  15            32  372            6              13          14         210   
  14            30   60           12               9          14         338   
  13            28  419            0               3          14          77   
  16            11  254           13               1          12         175   
  17            86   77            6               9          12         818   
  18            11  390            2               1          10          70   
  19            62   11           54               9          10         766   
  20            36  432           28               6          10         272   
  21            33  217           11               5           9         193   
  22            19   34            2              16           9         148   
  23            10  510            0               6           8          10   
  29             5  679            0               8           7          66   
  33            21  333           11               4           7          36   
  32            16  227           10               2           7         235   
  31            46  222            2              14           7         138   
  30            23  281            4               8           7          48   
  25            17  462            0               1           7          73   
  28            10   71            0               4           7          76   
  26             9  566            0               1           7          29   
  24            14  487            0               0           7          83   
  27            14    9            0               9           7          93   
  34             9  195            0               1           6          33   
  35            17    4            0              17           6          42   
  36            18  237            1              25           6          51   
  37            13  145            0               7           6          50   
  38            12  351            0               8           6          55   
  39            46  303            5               3           6         162   
  45            23    7           21               5           5         223   
  48            40   67            6               7           5         290   
  47            71  229            6               0           5         141   
  46            32  184           14               2           5         275   
  40             6  562            0               0           5          22   
  44             5  565            2               3           5          33   
  43             6  556            3               6           5          65   
  42            11  430            2               3           5          41   
  41            11  247            6              14           5  

**Discourse users sorted by likes given**

In [19]:
plot_user_data(users.head(50), 'likes_given')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             78    6          216             127         160        1374   
  1             90    3          197             116         135         988   
  9             81   30           77              17          24         324   
  19            62   11           54               9          10         766   
  10            58   99           43              19          22         533   
  11            30  485           42              17          18         246   
  20            36  432           28               6          10         272   
  45            23    7           21               5           5         223   
  5             71  116           21              51          44         610   
  6             77   73           17              30          35        1527   
  2             65   24           16              51          71         668   
  46            32  184           14               2           5         275   
  16            11  254           13               1          12         175   
  14            30   60           12               9          14         338   
  21            33  217           11               5           9         193   
  33            21  333           11               4           7          36   
  32            16  227           10               2           7         235   
  3             82    8           10              59          53         830   
  4             64   12            9              48          47         844   
  17            86   77            6               9          12         818   
  41            11  247            6              14           5         183   
  47            71  229            6               0           5         141   
  15            32  372            6              13          14         210   
  48            40   67            6               7           5         290   
  39            46  303            5               3           6         162   
  12            19  234            5               3          16          84   
  8             62  239            5               7          25         438   
  30            23  281            4               8           7          48   
  43             6  556            3               6           5          65   
  42            11  430            2               3           5          41   
  18            11  390            2               1          10          70   
  31            46  222            2              14           7         138   
  44             5  565            2               3           5          33   
  22            19   34            2              16           9         148   
  7             15  220            1              24          27         189   
  36            18  237            1              25           6          51   
  40             6  562            0               0           5          22   
  38            12  351            0               8           6          55   
  25            17  462            0               1           7          73   
  37            13  145            0               7           6          50   
  35            17    4            0              17           6          42   
  34             9  195            0               1           6          33   
  29             5  679            0               8           7          66   
  28            10   71            0               4           7          76   
  27            14    9            0               9           7          93   
  26             9  566            0               1           7          29   
  24            14  487            0               0           7          83   
  23            10  510            0               6           8          10   
  13            28  419            0               3          14  

**Discourse users sorted by likes received**

In [20]:
plot_user_data(users.head(50), 'likes_received')

Chart({
  data:     days_visited   id  likes_given  likes_received  post_count  posts_read  \
  0             78    6          216             127         160        1374   
  1             90    3          197             116         135         988   
  3             82    8           10              59          53         830   
  2             65   24           16              51          71         668   
  5             71  116           21              51          44         610   
  4             64   12            9              48          47         844   
  6             77   73           17              30          35        1527   
  36            18  237            1              25           6          51   
  7             15  220            1              24          27         189   
  10            58   99           43              19          22         533   
  11            30  485           42              17          18         246   
  35            17    4            0              17           6          42   
  9             81   30           77              17          24         324   
  22            19   34            2              16           9         148   
  31            46  222            2              14           7         138   
  41            11  247            6              14           5         183   
  15            32  372            6              13          14         210   
  14            30   60           12               9          14         338   
  17            86   77            6               9          12         818   
  19            62   11           54               9          10         766   
  27            14    9            0               9           7          93   
  30            23  281            4               8           7          48   
  29             5  679            0               8           7          66   
  38            12  351            0               8           6          55   
  37            13  145            0               7           6          50   
  8             62  239            5               7          25         438   
  48            40   67            6               7           5         290   
  23            10  510            0               6           8          10   
  20            36  432           28               6          10         272   
  43             6  556            3               6           5          65   
  45            23    7           21               5           5         223   
  21            33  217           11               5           9         193   
  33            21  333           11               4           7          36   
  49            48  298            0               4           5         317   
  28            10   71            0               4           7          76   
  39            46  303            5               3           6         162   
  42            11  430            2               3           5          41   
  44             5  565            2               3           5          33   
  13            28  419            0               3          14          77   
  12            19  234            5               3          16          84   
  32            16  227           10               2           7         235   
  46            32  184           14               2           5         275   
  34             9  195            0               1           6          33   
  26             9  566            0               1           7          29   
  18            11  390            2               1          10          70   
  16            11  254           13               1          12         175   
  25            17  462            0               1           7          73   
  24            14  487            0               0           7          83   
  40             6  562            0               0           5  

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [21]:
# Define colors we'll use for GitHub membership
import seaborn as sns
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_colors = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_colors]

In [22]:
from datetime import timedelta

orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/nteract](https://github.com/nteract)

In [23]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-05-17** to **2019-07-16**

In [24]:
responses = []
for org in github_orgs:
    query_prs = f"is:merged user:{org} archived:false closed:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
merged = pd.concat([ii.data for ii in responses])

Found 235 items, which will take 5 pages
Found 257 items, which will take 6 pages
Found 228 items, which will take 5 pages
Found 93 items, which will take 2 pages
Found 54 items, which will take 2 pages
Found 3 items, which will take 1 pages
Found 141 items, which will take 3 pages


In [25]:
# Add a PR-specific field for closed PRs
merged['closed_by'] = merged['mergedBy'].map(lambda a: a['login'])
merged['closed_by_url'] = merged['mergedBy'].map(lambda a: a['url'])

In [26]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)

## Commentors

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [27]:
alt.Chart(data=prs_by_repo, title="Merged PRs in the last 30 days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                           repo  author
  75          nteract                       hydrogen      40
  73          nteract                      bookstore      36
  78          nteract                        nteract      28
  80          nteract                      papermill      16
  79          nteract                     nteract.io       9
  74          nteract                       commuter       3
  81          nteract                      scrapbook       3
  77          nteract                 minimal-plotly       2
  83          nteract                  vega-embed-v3       2
  76          nteract                   kernel-relay       1
  82          nteract                           vdom       1
  58       jupyterlab                     jupyterlab     164
  62       jupyterlab                 jupyterlab-git      13
  69       jupyterlab              jupyterlab_server       9
  61       jupyterlab                jupyterlab-demo       8
  57       jupyterlab              jupyter-renderers       7
  60       jupyterlab       jupyterlab-data-explorer       6
  59       jupyterlab          jupyterlab-commenting       4
  72       jupyterlab             theme-cookiecutter       3
  56       jupyterlab      extension-cookiecutter-ts       2
  63       jupyterlab              jupyterlab-github       2
  67       jupyterlab                 jupyterlab-toc       2
  71       jupyterlab  scipy2019-jupyterlab-tutorial       2
  55       jupyterlab      extension-cookiecutter-js       1
  64       jupyterlab               jupyterlab-latex       1
  65       jupyterlab    jupyterlab-metadata-service       1
  66       jupyterlab           jupyterlab-telemetry       1
  68       jupyterlab            jupyterlab_pygments       1
  70       jupyterlab     mimerender-cookiecutter-ts       1
  43       jupyterhub            mybinder.org-deploy      91
  ..              ...                            ...     ...
  29  jupyter-widgets                      pythreejs       5
  27  jupyter-widgets             jupyterlab-sidecar       1
  28  jupyter-widgets                   midicontrols       1
  30  jupyter-widgets                     traittypes       1
  32  jupyter-widgets            widget-cookiecutter       1
  21          jupyter                       notebook      50
  19          jupyter                       nbgrader      43
  23          jupyter                    repo2docker      27
  8           jupyter             enterprise_gateway      25
  17          jupyter                      nbconvert      18
  24          jupyter                      telemetry      15
  13          jupyter                 jupyter_client      12
  10          jupyter                   jupyter-book      11
  11          jupyter                 jupyter-sphinx      11
  22          jupyter                      qtconsole       9
  7           jupyter                  docker-stacks       8
  15          jupyter                 jupyter_server       8
  14          jupyter                   jupyter_core       7
  12          jupyter              jupyter.github.io       6
  20          jupyter                       nbviewer       4
  9           jupyter                        jupyter       1
  16          jupyter                          nb2kg       1
  18          jupyter                         nbdime       1
  5           ipython                        ipython      43
  6           ipython                      traitlets       5
  3           ipython                      ipykernel       4
  4           ipython                    ipyparallel       2
  0   binder-examples                     data-quilt       1
  1   binder-examples             getting-data-xroot       1
  2   binder-examples                          latex       1
  
  [84 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['hydrogen', 'bookstore', 'nterac

### A list of merged PRs by project

In [28]:
from ipywidgets import widgets, HTML
from ipywidgets import HTML
from markdown import markdown 

In [29]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['username']
            user_url = pr['username_url']
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['closed_by']
            pr_closedby_url = pr['closed_by_url']
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

Authoring and merging stats by repo

In [30]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'username']).count()['author'].reset_index().rename(columns={'author': "authored"})
closed_by_repo = merged.groupby(['org', 'repo', 'closed_by']).count()['author'].reset_index().rename(columns={'author': "closed", "closed_by": "username"})

merged_stats = pd.merge(closed_by_repo, merged_by_repo, on=['org', 'repo', 'username'], how='outer')
merged_stats = pd.melt(merged_stats, id_vars=['org', 'repo', 'username'], var_name="kind", value_name="count")
merged_stats = merged_stats.replace(np.nan, 0)

In [31]:
for ii, (iorg, idata) in enumerate(merged_stats.groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='count',
        color='kind',
    )
    display(ch)

Chart({
  data:                  org                repo         username      kind  count
  0    binder-examples          data-quilt          betatim    closed    1.0
  1    binder-examples  getting-data-xroot        choldgraf    closed    1.0
  2    binder-examples               latex        choldgraf    closed    1.0
  179  binder-examples          data-quilt      matheusmota    closed    0.0
  180  binder-examples  getting-data-xroot  matthewfeickert    closed    0.0
  181  binder-examples               latex        yuvipanda    closed    0.0
  469  binder-examples          data-quilt          betatim  authored    0.0
  470  binder-examples  getting-data-xroot        choldgraf  authored    0.0
  471  binder-examples               latex        choldgraf  authored    0.0
  648  binder-examples          data-quilt      matheusmota  authored    1.0
  649  binder-examples  getting-data-xroot  matthewfeickert  authored    1.0
  650  binder-examples               latex        yuvipanda  authored    1.0,
  encoding: FacetedEncoding({
    color: Color({
      field: 'kind',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merges for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:          org         repo        username      kind  count
  3    ipython    ipykernel           minrk    closed    4.0
  4    ipython  ipyparallel           minrk    closed    2.0
  5    ipython      ipython         Carreau    closed   43.0
  6    ipython    traitlets        rmorshea    closed    5.0
  182  ipython    ipykernel         Chilipp    closed    0.0
  183  ipython    ipykernel         hroncok    closed    0.0
  184  ipython    ipykernel        steff456    closed    0.0
  185  ipython  ipyparallel           Jamim    closed    0.0
  186  ipython  ipyparallel          ellert    closed    0.0
  187  ipython      ipython  LucianaMarques    closed    0.0
  188  ipython      ipython      OriolAbril    closed    0.0
  189  ipython      ipython       Pragnya02    closed    0.0
  190  ipython      ipython            alok    closed    0.0
  191  ipython      ipython         anntzer    closed    0.0
  192  ipython      ipython        bashtage    closed    0.0
  193  ipython      ipython         blueyed    closed    0.0
  194  ipython      ipython  brandonwillard    closed    0.0
  195  ipython      ipython          daharn    closed    0.0
  196  ipython      ipython        eobrie16    closed    0.0
  197  ipython      ipython      gooverdian    closed    0.0
  198  ipython      ipython         hroncok    closed    0.0
  199  ipython      ipython           k-zen    closed    0.0
  200  ipython      ipython          kmaork    closed    0.0
  201  ipython      ipython      pierstitus    closed    0.0
  202  ipython      ipython           rgson    closed    0.0
  203  ipython      ipython         sunpoet    closed    0.0
  204  ipython      ipython      sysadmin75    closed    0.0
  205  ipython      ipython       tacaswell    closed    0.0
  206  ipython    traitlets           Jamim    closed    0.0
  207  ipython    traitlets    danielballan    closed    0.0
  ..       ...          ...             ...       ...    ...
  474  ipython      ipython         Carreau  authored   21.0
  475  ipython    traitlets        rmorshea  authored    0.0
  651  ipython    ipykernel         Chilipp  authored    1.0
  652  ipython    ipykernel         hroncok  authored    1.0
  653  ipython    ipykernel        steff456  authored    1.0
  654  ipython  ipyparallel           Jamim  authored    1.0
  655  ipython  ipyparallel          ellert  authored    1.0
  656  ipython      ipython  LucianaMarques  authored    2.0
  657  ipython      ipython      OriolAbril  authored    2.0
  658  ipython      ipython       Pragnya02  authored    1.0
  659  ipython      ipython            alok  authored    2.0
  660  ipython      ipython         anntzer  authored    1.0
  661  ipython      ipython        bashtage  authored    1.0
  662  ipython      ipython         blueyed  authored    1.0
  663  ipython      ipython  brandonwillard  authored    1.0
  664  ipython      ipython          daharn  authored    1.0
  665  ipython      ipython        eobrie16  authored    1.0
  666  ipython      ipython      gooverdian  authored    1.0
  667  ipython      ipython         hroncok  authored    1.0
  668  ipython      ipython           k-zen  authored    1.0
  669  ipython      ipython          kmaork  authored    1.0
  670  ipython      ipython      pierstitus  authored    1.0
  671  ipython      ipython           rgson  authored    1.0
  672  ipython      ipython         sunpoet  authored    1.0
  673  ipython      ipython      sysadmin75  authored    1.0
  674  ipython      ipython       tacaswell  authored    1.0
  675  ipython    traitlets           Jamim  authored    2.0
  676  ipython    traitlets    danielballan  authored    1.0
  677  ipython    traitlets   frenzymadness  authored    1.0
  678  ipython    traitlets     kevin-bates  authored    1.0
  
  [64 rows x 5 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'kind',
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({

Chart({
  data:          org                repo       username      kind  count
  7    jupyter       docker-stacks          minrk    closed    6.0
  8    jupyter       docker-stacks        parente    closed    2.0
  9    jupyter  enterprise_gateway      akchinSTC    closed    2.0
  10   jupyter  enterprise_gateway    kevin-bates    closed   17.0
  11   jupyter  enterprise_gateway       lresende    closed    6.0
  12   jupyter             jupyter        parente    closed    1.0
  13   jupyter        jupyter-book      choldgraf    closed   11.0
  14   jupyter      jupyter-sphinx  SylvainCorlay    closed    1.0
  15   jupyter      jupyter-sphinx       akhmerov    closed    9.0
  16   jupyter      jupyter-sphinx       jbweston    closed    1.0
  17   jupyter   jupyter.github.io           Ruv7    closed    1.0
  18   jupyter   jupyter.github.io         afshin    closed    1.0
  19   jupyter   jupyter.github.io      choldgraf    closed    1.0
  20   jupyter   jupyter.github.io      ellisonbg    closed    1.0
  21   jupyter   jupyter.github.io     jasongrout    closed    2.0
  22   jupyter      jupyter_client          MSeal    closed    1.0
  23   jupyter      jupyter_client  SylvainCorlay    closed    4.0
  24   jupyter      jupyter_client      blink1073    closed    1.0
  25   jupyter      jupyter_client          minrk    closed    1.0
  26   jupyter      jupyter_client         mpacer    closed    1.0
  27   jupyter      jupyter_client         rgbkrk    closed    4.0
  28   jupyter        jupyter_core          minrk    closed    1.0
  29   jupyter        jupyter_core         rgbkrk    closed    1.0
  30   jupyter        jupyter_core      takluyver    closed    5.0
  31   jupyter      jupyter_server  SylvainCorlay    closed    3.0
  32   jupyter      jupyter_server      blink1073    closed    1.0
  33   jupyter      jupyter_server         ivanov    closed    1.0
  34   jupyter      jupyter_server       rolweber    closed    3.0
  35   jupyter               nb2kg    kevin-bates    closed    1.0
  36   jupyter           nbconvert          MSeal    closed   13.0
  ..       ...                 ...            ...       ...    ...
  734  jupyter            notebook        betatim  authored    1.0
  735  jupyter            notebook    dalanicolai  authored    1.0
  736  jupyter            notebook        ednut15  authored    3.0
  737  jupyter            notebook    fabihaahmed  authored    3.0
  738  jupyter            notebook        gnestor  authored    1.0
  739  jupyter            notebook         hdtee1  authored    1.0
  740  jupyter            notebook  joshuazeltser  authored   16.0
  741  jupyter            notebook    kevin-bates  authored    3.0
  742  jupyter            notebook         mgeier  authored    1.0
  743  jupyter            notebook        minho42  authored    1.0
  744  jupyter            notebook       mtd91429  authored    1.0
  745  jupyter            notebook       saraedum  authored    1.0
  746  jupyter            notebook     ssanderson  authored    1.0
  747  jupyter            notebook       t-makaro  authored    1.0
  748  jupyter            notebook       taohan16  authored    1.0
  749  jupyter            notebook         tholoz  authored    1.0
  750  jupyter            notebook         tmetzl  authored    1.0
  751  jupyter            notebook     vibvibgyor  authored    1.0
  752  jupyter           qtconsole          ajtam  authored    1.0
  753  jupyter           qtconsole       dalthviz  authored    2.0
  754  jupyter           qtconsole       impact27  authored    1.0
  755  jupyter         repo2docker     Xarthisius  authored    1.0
  756  jupyter         repo2docker   consideRatio  authored    1.0
  757  jupyter         repo2docker   davidanthoff  authored    1.0
  758  jupyter         repo2docker     davidrpugh  authored    1.0
  759  jupyter         repo2docker      eexwhyzee  authored    1.0
  760  jupyter         repo2docker      fmaussion  authored    1.0
  761  jupyter         repo2docke

Chart({
  data:                  org                 repo          username      kind  count
  56   jupyter-widgets           ipyleaflet     SylvainCorlay    closed    3.0
  57   jupyter-widgets           ipyleaflet       martinRenou    closed   21.0
  58   jupyter-widgets           ipywidgets     SylvainCorlay    closed   20.0
  59   jupyter-widgets           ipywidgets        jasongrout    closed    7.0
  60   jupyter-widgets           ipywidgets           vidartf    closed    3.0
  61   jupyter-widgets   jupyterlab-sidecar     SylvainCorlay    closed    1.0
  62   jupyter-widgets         midicontrols        jasongrout    closed    1.0
  63   jupyter-widgets            pythreejs           vidartf    closed    5.0
  64   jupyter-widgets           traittypes     SylvainCorlay    closed    1.0
  65   jupyter-widgets             tutorial     SylvainCorlay    closed    1.0
  66   jupyter-widgets             tutorial   maartenbreddels    closed    1.0
  67   jupyter-widgets             tutorial       martinRenou    closed    4.0
  68   jupyter-widgets             tutorial           mwcraig    closed   24.0
  69   jupyter-widgets  widget-cookiecutter           vidartf    closed    1.0
  295  jupyter-widgets           ipyleaflet         Juanlu001    closed    0.0
  296  jupyter-widgets           ipyleaflet      VasavanThiru    closed    0.0
  297  jupyter-widgets           ipyleaflet          deeplook    closed    0.0
  298  jupyter-widgets           ipyleaflet             jmp75    closed    0.0
  299  jupyter-widgets           ipyleaflet             jtpio    closed    0.0
  300  jupyter-widgets           ipyleaflet         kgiacobbi    closed    0.0
  301  jupyter-widgets           ipyleaflet          mpdwulit    closed    0.0
  302  jupyter-widgets           ipywidgets             Jamim    closed    0.0
  303  jupyter-widgets           ipywidgets         Juanlu001    closed    0.0
  304  jupyter-widgets           ipywidgets          akhmerov    closed    0.0
  305  jupyter-widgets           ipywidgets             blois    closed    0.0
  306  jupyter-widgets           ipywidgets           edolley    closed    0.0
  307  jupyter-widgets           ipywidgets            ivanov    closed    0.0
  308  jupyter-widgets           ipywidgets             jtpio    closed    0.0
  309  jupyter-widgets           ipywidgets            lheagy    closed    0.0
  310  jupyter-widgets           ipywidgets       martinRenou    closed    0.0
  ..               ...                  ...               ...       ...    ...
  536  jupyter-widgets             tutorial       martinRenou  authored   17.0
  537  jupyter-widgets             tutorial           mwcraig  authored   10.0
  538  jupyter-widgets  widget-cookiecutter           vidartf  authored    0.0
  764  jupyter-widgets           ipyleaflet         Juanlu001  authored    1.0
  765  jupyter-widgets           ipyleaflet      VasavanThiru  authored    4.0
  766  jupyter-widgets           ipyleaflet          deeplook  authored    1.0
  767  jupyter-widgets           ipyleaflet             jmp75  authored    1.0
  768  jupyter-widgets           ipyleaflet             jtpio  authored    1.0
  769  jupyter-widgets           ipyleaflet         kgiacobbi  authored    5.0
  770  jupyter-widgets           ipyleaflet          mpdwulit  authored    1.0
  771  jupyter-widgets           ipywidgets             Jamim  authored    1.0
  772  jupyter-widgets           ipywidgets         Juanlu001  authored    1.0
  773  jupyter-widgets           ipywidgets          akhmerov  authored    1.0
  774  jupyter-widgets           ipywidgets             blois  authored    1.0
  775  jupyter-widgets           ipywidgets           edolley  authored    1.0
  776  jupyter-widgets           ipywidgets            ivanov  authored    1.0
  777  jupyter-widgets           ipywidgets             jtpio  authored    1.0
  778  jupyter-widgets           ipywidgets            lheagy  authored    1.0
  779  jupyter-widgets           ipy

Chart({
  data:             org                     repo        username      kind  count
  70   jupyterhub     alabaster-jupyterhub         betatim    closed    1.0
  71   jupyterhub     alabaster-jupyterhub       choldgraf    closed    8.0
  72   jupyterhub             batchspawner      mbmilligan    closed    7.0
  73   jupyterhub                   binder         betatim    closed    1.0
  74   jupyterhub                binderhub         betatim    closed   11.0
  75   jupyterhub                binderhub       choldgraf    closed    2.0
  76   jupyterhub                binderhub    consideRatio    closed    3.0
  77   jupyterhub                binderhub           minrk    closed    1.0
  78   jupyterhub                binderhub       sgibson91    closed    1.0
  79   jupyterhub                binderhub       yuvipanda    closed    1.0
  80   jupyterhub               chartpress           minrk    closed    1.0
  81   jupyterhub               chartpress       yuvipanda    closed    1.0
  82   jupyterhub            dockerspawner           minrk    closed    1.0
  83   jupyterhub     jupyter-server-proxy         betatim    closed    1.0
  84   jupyterhub     jupyter-server-proxy       yuvipanda    closed    2.0
  85   jupyterhub               jupyterhub         betatim    closed    3.0
  86   jupyterhub               jupyterhub           minrk    closed   10.0
  87   jupyterhub               jupyterhub       yuvipanda    closed    3.0
  88   jupyterhub    kerberosauthenticator          jcrist    closed    4.0
  89   jupyterhub         ltiauthenticator       yuvipanda    closed    1.0
  90   jupyterhub      mybinder.org-deploy         betatim    closed   78.0
  91   jupyterhub      mybinder.org-deploy       choldgraf    closed    4.0
  92   jupyterhub      mybinder.org-deploy    consideRatio    closed    1.0
  93   jupyterhub      mybinder.org-deploy          henchc    closed    5.0
  94   jupyterhub      mybinder.org-deploy           minrk    closed    2.0
  95   jupyterhub      mybinder.org-deploy       sgibson91    closed    1.0
  96   jupyterhub              nbgitpuller       choldgraf    closed    2.0
  97   jupyterhub              nbgitpuller       yuvipanda    closed    3.0
  98   jupyterhub           oauthenticator         betatim    closed    1.0
  99   jupyterhub           oauthenticator           minrk    closed    3.0
  ..          ...                      ...             ...       ...    ...
  824  jupyterhub           oauthenticator    DentonGentry  authored    1.0
  825  jupyterhub           oauthenticator          nifuki  authored    1.0
  826  jupyterhub           oauthenticator       sunnielyu  authored    1.0
  827  jupyterhub              sudospawner         lumbric  authored    1.0
  828  jupyterhub           systemdspawner       mortbauer  authored    1.0
  829  jupyterhub             team-compass         Zsailer  authored    2.0
  830  jupyterhub             team-compass          lheagy  authored    1.0
  831  jupyterhub             team-compass      ryanlovett  authored    1.0
  832  jupyterhub  the-littlest-jupyterhub         JuanCab  authored    1.0
  833  jupyterhub  the-littlest-jupyterhub     cornhundred  authored    1.0
  834  jupyterhub  the-littlest-jupyterhub           jtpio  authored    7.0
  835  jupyterhub  the-littlest-jupyterhub         lumbric  authored    1.0
  836  jupyterhub  the-littlest-jupyterhub            owah  authored    1.0
  837  jupyterhub  the-littlest-jupyterhub        pbugnion  authored    1.0
  838  jupyterhub  the-littlest-jupyterhub         staeiou  authored    2.0
  839  jupyterhub  the-littlest-jupyterhub        trallard  authored    1.0
  840  jupyterhub         tmpauthenticator         mohirio  authored    1.0
  841  jupyterhub            traefik-proxy  GeorgianaElena  authored    5.0
  842  jupyterhub              wrapspawner        cmd-ntrf  authored    2.0
  843  jupyterhub              yarnspawner  reallyhappypig  authored    1.0
  844  jupyterhub   zero-to-jupyte

Chart({
  data:             org                           repo         username      kind  \
  117  jupyterlab      extension-cookiecutter-js            jtpio    closed   
  118  jupyterlab      extension-cookiecutter-ts        blink1073    closed   
  119  jupyterlab              jupyter-renderers          gnestor    closed   
  120  jupyterlab              jupyter-renderers       ian-r-rose    closed   
  121  jupyterlab                     jupyterlab           afshin    closed   
  122  jupyterlab                     jupyterlab         aschlaep    closed   
  123  jupyterlab                     jupyterlab        blink1073    closed   
  124  jupyterlab                     jupyterlab        ellisonbg    closed   
  125  jupyterlab                     jupyterlab          gnestor    closed   
  126  jupyterlab                     jupyterlab       ian-r-rose    closed   
  127  jupyterlab                     jupyterlab           ivanov    closed   
  128  jupyterlab                     jupyterlab       jasongrout    closed   
  129  jupyterlab                     jupyterlab            jtpio    closed   
  130  jupyterlab                     jupyterlab   saulshanabrook    closed   
  131  jupyterlab                     jupyterlab       telamonian    closed   
  132  jupyterlab                     jupyterlab          vidartf    closed   
  133  jupyterlab          jupyterlab-commenting           acu192    closed   
  134  jupyterlab          jupyterlab-commenting           hoo761    closed   
  135  jupyterlab       jupyterlab-data-explorer   saulshanabrook    closed   
  136  jupyterlab                jupyterlab-demo      dharmaquark    closed   
  137  jupyterlab                jupyterlab-demo       ian-r-rose    closed   
  138  jupyterlab                jupyterlab-demo       jasongrout    closed   
  139  jupyterlab                 jupyterlab-git            dleen    closed   
  140  jupyterlab                 jupyterlab-git       jaipreet-s    closed   
  141  jupyterlab                 jupyterlab-git   saulshanabrook    closed   
  142  jupyterlab              jupyterlab-github       ian-r-rose    closed   
  143  jupyterlab               jupyterlab-latex       ian-r-rose    closed   
  144  jupyterlab    jupyterlab-metadata-service           acu192    closed   
  145  jupyterlab           jupyterlab-telemetry          Zsailer    closed   
  146  jupyterlab                 jupyterlab-toc       ian-r-rose    closed   
  ..          ...                            ...              ...       ...   
  885  jupyterlab                     jupyterlab      parmentelat  authored   
  886  jupyterlab                     jupyterlab          reevejd  authored   
  887  jupyterlab                     jupyterlab         tobiasjj  authored   
  888  jupyterlab                     jupyterlab          tslaton  authored   
  889  jupyterlab                     jupyterlab        yuvipanda  authored   
  890  jupyterlab                     jupyterlab          zerline  authored   
  891  jupyterlab          jupyterlab-commenting       dependabot  authored   
  892  jupyterlab       jupyterlab-data-explorer       dependabot  authored   
  893  jupyterlab       jupyterlab-data-explorer       telamonian  authored   
  894  jupyterlab                jupyterlab-demo    SylvainCorlay  authored   
  895  jupyterlab                jupyterlab-demo        TheBear44  authored   
  896  jupyterlab                 jupyterlab-git    danielgruesso  authored   
  897  jupyterlab                 jupyterlab-git       dependabot  authored   
  898  jupyterlab                 jupyterlab-git       fcollonval  authored   
  899  jupyterlab                 jupyterlab-git     neelamgehlot  authored   
  900  jupyterlab                 jupyterlab-git        urihoenig  authored   
  901  jupyterlab              jupyterlab-github       dependabot  authored   
  902  jupyterlab               jupyterlab-latex       dependabot  authored   
  903  jupyterlab           jupyterl

Chart({
  data:          org            repo        username      kind  count
  154  nteract       bookstore           MSeal    closed    3.0
  155  nteract       bookstore          mpacer    closed    8.0
  156  nteract       bookstore          rgbkrk    closed    6.0
  157  nteract       bookstore        willingc    closed   19.0
  158  nteract        commuter    captainsafia    closed    3.0
  159  nteract        hydrogen      BenRussert    closed   22.0
  160  nteract        hydrogen        aviatesk    closed   10.0
  161  nteract        hydrogen        renovate    closed    4.0
  162  nteract        hydrogen  wadethestealth    closed    4.0
  163  nteract    kernel-relay    captainsafia    closed    1.0
  164  nteract  minimal-plotly          rgbkrk    closed    2.0
  165  nteract         nteract      BenRussert    closed    8.0
  166  nteract         nteract  CrystallineCat    closed    1.0
  167  nteract         nteract    captainsafia    closed    8.0
  168  nteract         nteract          emeeks    closed    1.0
  169  nteract         nteract          rgbkrk    closed   10.0
  170  nteract      nteract.io        renovate    closed    8.0
  171  nteract      nteract.io          rgbkrk    closed    1.0
  172  nteract       papermill           MSeal    closed   11.0
  173  nteract       papermill          rgbkrk    closed    4.0
  174  nteract       papermill        willingc    closed    1.0
  175  nteract       scrapbook           MSeal    closed    2.0
  176  nteract       scrapbook          rgbkrk    closed    1.0
  177  nteract            vdom         gnestor    closed    1.0
  178  nteract   vega-embed-v3  CrystallineCat    closed    2.0
  446  nteract        commuter      dependabot    closed    0.0
  447  nteract        commuter          taniki    closed    0.0
  448  nteract        hydrogen     JeremyStepp    closed    0.0
  449  nteract        hydrogen        hoishing    closed    0.0
  450  nteract        hydrogen       jwrussert    closed    0.0
  ..       ...             ...             ...       ...    ...
  641  nteract       papermill           MSeal  authored    2.0
  642  nteract       papermill          rgbkrk  authored    1.0
  643  nteract       papermill        willingc  authored    0.0
  644  nteract       scrapbook           MSeal  authored    1.0
  645  nteract       scrapbook          rgbkrk  authored    0.0
  646  nteract            vdom         gnestor  authored    1.0
  647  nteract   vega-embed-v3  CrystallineCat  authored    2.0
  915  nteract        commuter      dependabot  authored    2.0
  916  nteract        commuter          taniki  authored    1.0
  917  nteract        hydrogen     JeremyStepp  authored    1.0
  918  nteract        hydrogen        hoishing  authored    1.0
  919  nteract        hydrogen       jwrussert  authored    1.0
  920  nteract    kernel-relay      BenRussert  authored    1.0
  921  nteract  minimal-plotly        renovate  authored    2.0
  922  nteract         nteract         and0111  authored    2.0
  923  nteract         nteract      dylanross3  authored    1.0
  924  nteract         nteract        renovate  authored    6.0
  925  nteract         nteract        rganardi  authored    2.0
  926  nteract         nteract  wadethestealth  authored    1.0
  927  nteract       papermill     RoelBrouwer  authored    1.0
  928  nteract       papermill  VincentBLortie  authored    1.0
  929  nteract       papermill   aabdullah-bos  authored    1.0
  930  nteract       papermill       casperdcl  authored    3.0
  931  nteract       papermill    duarteocarmo  authored    3.0
  932  nteract       papermill        khourhin  authored    1.0
  933  nteract       papermill          mwouts  authored    1.0
  934  nteract       papermill        trallard  authored    1.0
  935  nteract       papermill          vizhur  authored    1.0
  936  nteract       scrapbook      matthiasdv  authored    1.0
  937  nteract       scrapbook        trallard  authored    1.0
  
  [96 rows x 5 

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [32]:
responses = []
for org in github_orgs:
    query_prs = f"is:open is:issue user:{org} archived:false created:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
created = pd.concat([ii.data for ii in responses])
created['n_comments'] = created['comments'].map(lambda a: len(a['edges']))

Found 197 items, which will take 4 pages
Found 190 items, which will take 4 pages
Found 200 items, which will take 4 pages
Found 51 items, which will take 2 pages
Found 41 items, which will take 1 pages
Found 2 items, which will take 1 pages
Found 84 items, which will take 2 pages


In [33]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                           repo  number  \
  0   binder-examples                     demo-julia       1   
  1   binder-examples                   jupyter-rise       1   
  2           ipython                      ipykernel       6   
  3           ipython                    ipyparallel       3   
  4           ipython                        ipython      31   
  5           ipython               ipython-in-depth       1   
  6           jupyter                  accessibility       1   
  7           jupyter                         design       1   
  8           jupyter                  docker-stacks      13   
  9           jupyter             enterprise_gateway       5   
  10          jupyter                        jupyter       3   
  11          jupyter                   jupyter-book      20   
  12          jupyter                  jupyter-drive       1   
  13          jupyter                 jupyter-sphinx       7   
  14          jupyter                 jupyter_client       6   
  15          jupyter                   jupyter_core       1   
  16          jupyter            jupyter_kernel_test       1   
  17          jupyter                 jupyter_server       8   
  18          jupyter                 kernel_gateway       1   
  19          jupyter                          nb2kg       1   
  20          jupyter                      nbconvert      12   
  21          jupyter                         nbdime       6   
  22          jupyter                       nbgrader      24   
  23          jupyter                       nbviewer       3   
  24          jupyter                       notebook      53   
  25          jupyter                      qtconsole       2   
  26          jupyter                    repo2docker      20   
  27          jupyter                       testpath       1   
  28  jupyter-widgets                     ipyleaflet       6   
  29  jupyter-widgets                     ipywidgets      35   
  ..              ...                            ...     ...   
  59       jupyterhub         zero-to-jupyterhub-k8s      23   
  60       jupyterlab      extension-cookiecutter-ts       1   
  61       jupyterlab              jupyter-renderers       2   
  62       jupyterlab                     jupyterlab     150   
  63       jupyterlab            jupyterlab-celltags       3   
  64       jupyterlab          jupyterlab-commenting      12   
  65       jupyterlab       jupyterlab-data-explorer      15   
  66       jupyterlab                 jupyterlab-git       6   
  67       jupyterlab        jupyterlab-google-drive       1   
  68       jupyterlab    jupyterlab-metadata-service       1   
  69       jupyterlab           jupyterlab-telemetry       2   
  70       jupyterlab                 jupyterlab-toc       1   
  71       jupyterlab                jupyterlab_apod       1   
  72       jupyterlab                 jupyterlab_app       1   
  73       jupyterlab  scipy2018-jupyterlab-tutorial       1   
  74       jupyterlab                   team-compass       3   
  75          nteract                      bookstore      13   
  76          nteract                    coffee_boat       1   
  77          nteract                       commuter       1   
  78          nteract             commuter-on-glitch       1   
  79          nteract                             dx       3   
  80          nteract                       hydrogen      15   
  81          nteract                meeting-minutes       1   
  82          nteract                 minimal-plotly       1   
  83          nteract                        nteract      23   
  84          nteract                     nteract.io       1   
  85          nteract                      papermill      12   
  86          nteract                      scrapbook       3   
  87          nteract                       semiotic       8   
  88          nteract                           vdom       1   
  
               

In [34]:
responses = []
for org in github_orgs:
    query_prs = f"is:issue user:{org} closed:{start_date}..{end_date}"
    ghq = GitHubGraphQlQuery(query_prs)
    ghq.request()
    if ghq.data is None:
        continue
    responses.append(ghq)
    
closed = pd.concat([ii.data for ii in responses])
closed['n_comments'] = closed['comments'].map(lambda a: len(a['edges']))

Found 149 items, which will take 3 pages
Found 224 items, which will take 5 pages
Found 216 items, which will take 5 pages
Found 52 items, which will take 2 pages
Found 32 items, which will take 1 pages
Found 1 items, which will take 1 pages
Found 170 items, which will take 4 pages


In [35]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0   binder-examples                      bokeh       1   
  1           ipython                  ipykernel       2   
  2           ipython                    ipython      28   
  3           ipython                  traitlets       2   
  4           jupyter              atom-notebook       1   
  5           jupyter              docker-stacks       7   
  6           jupyter         enterprise_gateway      22   
  7           jupyter                    jupyter       3   
  8           jupyter               jupyter-book      12   
  9           jupyter             jupyter-sphinx      13   
  10          jupyter          jupyter.github.io       1   
  11          jupyter             jupyter_client       4   
  12          jupyter               jupyter_core       9   
  13          jupyter             jupyter_server       8   
  14          jupyter                      nb2kg       1   
  15          jupyter                  nbconvert      19   
  16          jupyter                     nbdime       1   
  17          jupyter                   nbgrader      39   
  18          jupyter                   nbviewer       4   
  19          jupyter                   notebook      54   
  20          jupyter                  qtconsole       6   
  21          jupyter                repo2docker      16   
  22          jupyter                  telemetry       3   
  23          jupyter                  terminado       1   
  24  jupyter-widgets                 ipyleaflet      14   
  25  jupyter-widgets                 ipywidgets      30   
  26  jupyter-widgets         jupyterlab-sidecar       1   
  27  jupyter-widgets               midicontrols       1   
  28  jupyter-widgets                  pythreejs       3   
  29  jupyter-widgets                   tutorial       3   
  ..              ...                        ...     ...   
  36       jupyterhub   jupyterhub-deploy-docker       1   
  37       jupyterhub      kerberosauthenticator       1   
  38       jupyterhub                kubespawner       2   
  39       jupyterhub          ldapauthenticator       1   
  40       jupyterhub        mybinder.org-deploy      11   
  41       jupyterhub                nbgitpuller      10   
  42       jupyterhub             oauthenticator       1   
  43       jupyterhub               team-compass      11   
  44       jupyterhub    the-littlest-jupyterhub      49   
  45       jupyterhub           tmpauthenticator       1   
  46       jupyterhub              traefik-proxy       5   
  47       jupyterhub                yarnspawner       1   
  48       jupyterhub     zero-to-jupyterhub-k8s      15   
  49       jupyterlab  extension-cookiecutter-js       1   
  50       jupyterlab          jupyter-renderers       7   
  51       jupyterlab                 jupyterlab     183   
  52       jupyterlab   jupyterlab-data-explorer       7   
  53       jupyterlab            jupyterlab-demo       2   
  54       jupyterlab             jupyterlab-git      12   
  55       jupyterlab          jupyterlab-github       1   
  56       jupyterlab             jupyterlab-toc       2   
  57       jupyterlab          jupyterlab_server       1   
  58          nteract                  bookstore      11   
  59          nteract                   commuter       1   
  60          nteract                   hydrogen      99   
  61          nteract                    nteract      40   
  62          nteract                  papermill      10   
  63          nteract                  scrapbook       1   
  64          nteract                   semiotic       5   
  65          nteract                       vdom       3   
  
                                  org/repo  
  0                  binder-examples/bokeh  
  1                      ipython/ipykernel  
  2                        ipython/ipython  
  3                      ipython/traitlets  
  4                  jupyter/atom-note

### A list of recent issues

In [36]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(created.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for (org, repo), issues in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, issue in issues.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['username']
            user_url = issue['username_url']
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)
    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 60 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [37]:
comments = []
for items in [created, merged, closed]:
    for ii, irow in items.iterrows():
        icomments = [ii['node'] for ii in irow['comments']['edges']]
        icomments = pd.DataFrame(icomments)
        icomments['repo'] = irow['repo']
        icomments['org'] = irow['org']
        comments.append(icomments)
comments = pd.concat(comments, sort=False)
comments = comments.query("createdAt > @start_date and createdAt < @end_date")
comments['username'] = comments['author'].map(lambda a: a['login'])
comments['username_url'] = comments['author'].map(lambda a: a['url'])

In [38]:
commentors = (
    comments
    .groupby(['org', 'repo', 'username', 'authorAssociation'])
    .count()['author']
    .reset_index()
    .sort_values(['org', 'author'], ascending=False)
)

In [39]:
n_plot = 50
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='username',
        y='author',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    display(ch)

Chart({
  data:                org                repo             username authorAssociation  \
  0  binder-examples               bokeh            choldgraf            MEMBER   
  1  binder-examples               bokeh  mathematicalmichael              NONE   
  2  binder-examples          data-quilt              betatim            MEMBER   
  3  binder-examples          data-quilt          matheusmota              NONE   
  4  binder-examples          demo-julia            choldgraf            MEMBER   
  5  binder-examples  getting-data-xroot            choldgraf            MEMBER   
  6  binder-examples  getting-data-xroot      matthewfeickert       CONTRIBUTOR   
  7  binder-examples        jupyter-rise            choldgraf            MEMBER   
  8  binder-examples        jupyter-rise  mathematicalmichael              NONE   
  
     author  
  0       2  
  1       1  
  2       1  
  3       1  
  4       1  
  5       1  
  6       1  
  7       1  
  8       1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'username',
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 commentors for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:         org         repo        username authorAssociation  author
  18  ipython      ipython         Carreau            MEMBER      75
  12  ipython    ipykernel         hroncok       CONTRIBUTOR       7
  63  ipython    traitlets        rmorshea      COLLABORATOR       6
  14  ipython    ipykernel           minrk            MEMBER       5
  19  ipython      ipython  LucianaMarques       CONTRIBUTOR       5
  11  ipython    ipykernel      ccordoba12       CONTRIBUTOR       4
  33  ipython      ipython         efiring       CONTRIBUTOR       4
  59  ipython    traitlets     kevin-bates       CONTRIBUTOR       3
  9   ipython    ipykernel         Carreau            MEMBER       2
  16  ipython  ipyparallel           minrk            MEMBER       2
  24  ipython      ipython         blueyed       CONTRIBUTOR       2
  31  ipython      ipython          dlukes              NONE       2
  35  ipython      ipython        fehiepsi              NONE       2
  40  ipython      ipython      jamesmyatt              NONE       2
  41  ipython      ipython        jfhbrook              NONE       2
  46  ipython      ipython          kmaork       CONTRIBUTOR       2
  48  ipython      ipython         mlucool              NONE       2
  49  ipython      ipython    sanketShetty              NONE       2
  52  ipython      ipython       tacaswell       CONTRIBUTOR       2
  54  ipython      ipython      thrasibule              NONE       2
  55  ipython      ipython         wenjuno              NONE       2
  57  ipython    traitlets           Jamim       CONTRIBUTOR       2
  62  ipython    traitlets          rgbkrk            MEMBER       2
  10  ipython    ipykernel         Chilipp       CONTRIBUTOR       1
  13  ipython    ipykernel      jamesmyatt              NONE       1
  15  ipython  ipyparallel           Jamim       CONTRIBUTOR       1
  17  ipython      ipython    Benjamin-Lee              NONE       1
  20  ipython      ipython      OriolAbril       CONTRIBUTOR       1
  21  ipython      ipython        aakhmetz              NONE       1
  22  ipython      ipython            alok       CONTRIBUTOR       1
  23  ipython      ipython         anntzer       CONTRIBUTOR       1
  25  ipython      ipython  brandonwillard       CONTRIBUTOR       1
  26  ipython      ipython    brunobeltran              NONE       1
  27  ipython      ipython      ccordoba12       CONTRIBUTOR       1
  28  ipython      ipython         csheaff              NONE       1
  29  ipython      ipython          daharn       CONTRIBUTOR       1
  30  ipython      ipython         dickmao              NONE       1
  32  ipython      ipython     drunkwcodes              NONE       1
  34  ipython      ipython  farisachugthai              NONE       1
  36  ipython      ipython       gdhameeja              NONE       1
  37  ipython      ipython        gpotter2       CONTRIBUTOR       1
  38  ipython      ipython         hroncok       CONTRIBUTOR       1
  39  ipython      ipython      ikizhvatov              NONE       1
  42  ipython      ipython       jmansilla              NONE       1
  43  ipython      ipython  joaomarcosgris              NONE       1
  44  ipython      ipython        jseabold       CONTRIBUTOR       1
  45  ipython      ipython           kernc              NONE       1
  47  ipython      ipython           kmtac              NONE       1
  50  ipython      ipython        stonebig       CONTRIBUTOR       1
  51  ipython      ipython         sunpoet       CONTRIBUTOR       1,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      

Chart({
  data:          org                repo       username authorAssociation  author
  195  jupyter            nbgrader       jhamrick            MEMBER      98
  288  jupyter         repo2docker        betatim            MEMBER      78
  267  jupyter            notebook      takluyver            MEMBER      68
  88   jupyter  enterprise_gateway    kevin-bates            MEMBER      59
  102  jupyter        jupyter-book      choldgraf            MEMBER      48
  163  jupyter           nbconvert          MSeal            MEMBER      40
  116  jupyter      jupyter-sphinx       akhmerov            MEMBER      36
  310  jupyter         repo2docker      yuvipanda            MEMBER      30
  150  jupyter      jupyter_server        Zsailer            MEMBER      28
  291  jupyter         repo2docker   consideRatio      COLLABORATOR      22
  311  jupyter           telemetry        Zsailer            MEMBER      20
  131  jupyter      jupyter_client  SylvainCorlay            MEMBER      19
  314  jupyter           telemetry      yuvipanda            MEMBER      17
  175  jupyter           nbconvert       t-makaro      COLLABORATOR      16
  199  jupyter            nbgrader        nthiery       CONTRIBUTOR      16
  203  jupyter            nbgrader        rkdarst       CONTRIBUTOR      16
  111  jupyter        jupyter-book    psychemedia              NONE      15
  211  jupyter            nbviewer       krinsman       CONTRIBUTOR      15
  275  jupyter           qtconsole     ccordoba12      COLLABORATOR      15
  149  jupyter      jupyter_server  SylvainCorlay            MEMBER      14
  300  jupyter         repo2docker          minrk            MEMBER      14
  312  jupyter           telemetry     jaipreet-s      COLLABORATOR      14
  84   jupyter  enterprise_gateway       echarles       CONTRIBUTOR      13
  130  jupyter      jupyter_client          MSeal            MEMBER      12
  156  jupyter      jupyter_server       rolweber      COLLABORATOR      12
  290  jupyter         repo2docker      choldgraf            MEMBER      12
  92   jupyter  enterprise_gateway       suryag10              NONE      11
  251  jupyter            notebook  joshuazeltser       CONTRIBUTOR      11
  265  jupyter            notebook     ssanderson       CONTRIBUTOR      10
  292  jupyter         repo2docker   davidanthoff       CONTRIBUTOR      10
  115  jupyter      jupyter-sphinx  SylvainCorlay            MEMBER       9
  147  jupyter        jupyter_core      takluyver            MEMBER       9
  185  jupyter            nbgrader        Sefriol              NONE       9
  202  jupyter            nbgrader    psychemedia              NONE       9
  104  jupyter        jupyter-book        emdupre      COLLABORATOR       8
  258  jupyter            notebook       lresende            MEMBER       8
  76   jupyter       docker-stacks        parente            MEMBER       7
  182  jupyter              nbdime        vidartf      COLLABORATOR       7
  190  jupyter            nbgrader        dsblank            MEMBER       7
  191  jupyter            nbgrader     gertingold       CONTRIBUTOR       7
  85   jupyter  enterprise_gateway         esevan      COLLABORATOR       6
  90   jupyter  enterprise_gateway      michzimny       CONTRIBUTOR       6
  119  jupyter      jupyter-sphinx   hugokerstens       CONTRIBUTOR       6
  142  jupyter      jupyter_client         rgbkrk            MEMBER       6
  213  jupyter            nbviewer        parente            MEMBER       6
  293  jupyter         repo2docker     davidrpugh       CONTRIBUTOR       6
  80   jupyter  enterprise_gateway       IMAM9AIS       CONTRIBUTOR       5
  98   jupyter             jupyter        parente            MEMBER       5
  100  jupyter        jupyter-book          BENR0              NONE       5
  103  jupyter        jupyter-book      cpjobling              NONE       5,
  encoding: FacetedEncoding({
    color: Color({
      field: 'authorAssociation',
      scale: Scale({
     

Chart({
  data:                  org                repo          username authorAssociation  \
  337  jupyter-widgets          ipywidgets     SylvainCorlay            MEMBER   
  347  jupyter-widgets          ipywidgets        jasongrout            MEMBER   
  325  jupyter-widgets          ipyleaflet       martinRenou            MEMBER   
  360  jupyter-widgets          ipywidgets           vidartf            MEMBER   
  380  jupyter-widgets            tutorial           mwcraig       CONTRIBUTOR   
  373  jupyter-widgets           pythreejs           vidartf            MEMBER   
  345  jupyter-widgets          ipywidgets             hainm              NONE   
  328  jupyter-widgets          ipyleaflet          stonebig       CONTRIBUTOR   
  379  jupyter-widgets            tutorial       martinRenou            MEMBER   
  353  jupyter-widgets          ipywidgets           mwcraig       CONTRIBUTOR   
  324  jupyter-widgets          ipyleaflet         kgiacobbi       CONTRIBUTOR   
  336  jupyter-widgets          ipywidgets         Ricyteach       CONTRIBUTOR   
  368  jupyter-widgets           pythreejs          aliddell              NONE   
  323  jupyter-widgets          ipyleaflet             jtpio       CONTRIBUTOR   
  349  jupyter-widgets          ipywidgets           kafonek              NONE   
  350  jupyter-widgets          ipywidgets   maartenbreddels            MEMBER   
  356  jupyter-widgets          ipywidgets        rskabelund              NONE   
  378  jupyter-widgets            tutorial   maartenbreddels            MEMBER   
  342  jupyter-widgets          ipywidgets         choldgraf              NONE   
  357  jupyter-widgets          ipywidgets        ssanderson       CONTRIBUTOR   
  364  jupyter-widgets  jupyterlab-sidecar        doutriaux1              NONE   
  371  jupyter-widgets           pythreejs          stonebig              NONE   
  320  jupyter-widgets          ipyleaflet          chbrandt              NONE   
  321  jupyter-widgets          ipyleaflet          deeplook       CONTRIBUTOR   
  322  jupyter-widgets          ipyleaflet       dwr-psandhu              NONE   
  330  jupyter-widgets          ipywidgets   DanielAristidou              NONE   
  335  jupyter-widgets          ipywidgets         Liwellyen              NONE   
  340  jupyter-widgets          ipywidgets         beasteers              NONE   
  358  jupyter-widgets          ipywidgets          stonebig       CONTRIBUTOR   
  362  jupyter-widgets  jupyterlab-sidecar     SylvainCorlay            MEMBER   
  366  jupyter-widgets        midicontrols        jasongrout            MEMBER   
  381  jupyter-widgets            tutorial           vidartf            MEMBER   
  318  jupyter-widgets          ipyleaflet         Juanlu001       CONTRIBUTOR   
  319  jupyter-widgets          ipyleaflet     SylvainCorlay            MEMBER   
  326  jupyter-widgets          ipyleaflet          mpdwulit       CONTRIBUTOR   
  327  jupyter-widgets          ipyleaflet       paulathomas              NONE   
  329  jupyter-widgets          ipywidgets        DYSpider13              NONE   
  331  jupyter-widgets          ipywidgets          DevFabrx              NONE   
  332  jupyter-widgets          ipywidgets    IlyaKazakevich       CONTRIBUTOR   
  333  jupyter-widgets          ipywidgets         JohnIssue              NONE   
  334  jupyter-widgets          ipywidgets         Juanlu001       CONTRIBUTOR   
  338  jupyter-widgets          ipywidgets          akhmerov       CONTRIBUTOR   
  339  jupyter-widgets          ipywidgets    artur-trzesiok              NONE   
  341  jupyter-widgets          ipywidgets            bybyte              NONE   
  343  jupyter-widgets          ipywidgets       cornhundred       CONTRIBUTOR   
  344  jupyter-widgets          ipywidgets          dazzag24              NONE   
  346  jupyter-widgets          ipywidgets            ivanov       CONTRIBUTOR   
  348  jupyter-widgets          ipywidgets       j

Chart({
  data:             org                     repo         username authorAssociation  \
  572  jupyterhub  the-littlest-jupyterhub        yuvipanda            MEMBER   
  492  jupyterhub      mybinder.org-deploy          betatim            MEMBER   
  394  jupyterhub                binderhub          betatim            MEMBER   
  532  jupyterhub             team-compass          betatim            MEMBER   
  533  jupyterhub             team-compass        choldgraf            MEMBER   
  512  jupyterhub              nbgitpuller        yuvipanda            MEMBER   
  593  jupyterhub   zero-to-jupyterhub-k8s     consideRatio            MEMBER   
  468  jupyterhub               jupyterhub          ryogesh       CONTRIBUTOR   
  501  jupyterhub      mybinder.org-deploy      mael-le-gal            MEMBER   
  494  jupyterhub      mybinder.org-deploy        choldgraf            MEMBER   
  548  jupyterhub  the-littlest-jupyterhub   GeorgianaElena      COLLABORATOR   
  389  jupyterhub             batchspawner          rkdarst       CONTRIBUTOR   
  542  jupyterhub             team-compass        sgibson91            MEMBER   
  396  jupyterhub                binderhub        choldgraf            MEMBER   
  506  jupyterhub              nbgitpuller        choldgraf            MEMBER   
  578  jupyterhub            traefik-proxy            minrk            MEMBER   
  397  jupyterhub                binderhub     consideRatio            MEMBER   
  462  jupyterhub               jupyterhub            minrk            MEMBER   
  504  jupyterhub      mybinder.org-deploy        yuvipanda            MEMBER   
  509  jupyterhub              nbgitpuller      parmentelat       CONTRIBUTOR   
  540  jupyterhub             team-compass            minrk            MEMBER   
  408  jupyterhub                binderhub        sgibson91            MEMBER   
  496  jupyterhub      mybinder.org-deploy           henchc            MEMBER   
  544  jupyterhub             team-compass        yuvipanda            MEMBER   
  477  jupyterhub    kerberosauthenticator           jcrist            MEMBER   
  576  jupyterhub            traefik-proxy   GeorgianaElena      COLLABORATOR   
  601  jupyterhub   zero-to-jupyterhub-k8s           manics       CONTRIBUTOR   
  432  jupyterhub               jupyterhub   GeorgianaElena       CONTRIBUTOR   
  436  jupyterhub               jupyterhub  RandyBetancourt              NONE   
  471  jupyterhub               jupyterhub        vilhelmen       CONTRIBUTOR   
  481  jupyterhub              kubespawner     consideRatio            MEMBER   
  599  jupyterhub   zero-to-jupyterhub-k8s         joshbode       CONTRIBUTOR   
  387  jupyterhub             batchspawner       mbmilligan            MEMBER   
  473  jupyterhub               jupyterhub        yuvipanda            MEMBER   
  502  jupyterhub      mybinder.org-deploy            minrk            MEMBER   
  521  jupyterhub           oauthenticator            minrk            MEMBER   
  539  jupyterhub             team-compass      mael-le-gal            MEMBER   
  543  jupyterhub             team-compass         willingc            MEMBER   
  545  jupyterhub  the-littlest-jupyterhub      CyborgDroid              NONE   
  405  jupyterhub                binderhub         memeplex              NONE   
  447  jupyterhub               jupyterhub      brayanrodpr              NONE   
  483  jupyterhub              kubespawner           gsemet       CONTRIBUTOR   
  600  jupyterhub   zero-to-jupyterhub-k8s        lakshaykc              NONE   
  399  jupyterhub                binderhub          jhamman       CONTRIBUTOR   
  406  jupyterhub                binderhub            minrk            MEMBER   
  410  jupyterhub                binderhub        yuvipanda            MEMBER   
  452  jupyterhub               jupyterhub          eylenth              NONE   
  457  jupyterhub               jupyterhub        lakshaykc              NONE   
  458  jupyterhub

Chart({
  data:             org                      repo             username  \
  706  jupyterlab                jupyterlab           jasongrout   
  658  jupyterlab                jupyterlab            blink1073   
  708  jupyterlab                jupyterlab  jupyterlab-dev-mode   
  700  jupyterlab                jupyterlab           ian-r-rose   
  762  jupyterlab                jupyterlab              vidartf   
  745  jupyterlab                jupyterlab       saulshanabrook   
  752  jupyterlab                jupyterlab           telamonian   
  684  jupyterlab                jupyterlab            ellisonbg   
  643  jupyterlab                jupyterlab               afshin   
  622  jupyterlab                jupyterlab               BoPeng   
  785  jupyterlab            jupyterlab-git           jaipreet-s   
  753  jupyterlab                jupyterlab            tgeorgeux   
  649  jupyterlab                jupyterlab             aschlaep   
  772  jupyterlab  jupyterlab-data-explorer       saulshanabrook   
  679  jupyterlab                jupyterlab             domoritz   
  757  jupyterlab                jupyterlab              tslaton   
  704  jupyterlab                jupyterlab              jakevdp   
  707  jupyterlab                jupyterlab                jtpio   
  716  jupyterlab                jupyterlab           krassowski   
  640  jupyterlab                jupyterlab              Zsailer   
  682  jupyterlab                jupyterlab         duarteocarmo   
  764  jupyterlab                jupyterlab         vtrokhymenko   
  777  jupyterlab           jupyterlab-demo           ian-r-rose   
  778  jupyterlab           jupyterlab-demo           jasongrout   
  695  jupyterlab                jupyterlab              gnestor   
  701  jupyterlab                jupyterlab              ibdafna   
  729  jupyterlab                jupyterlab           michaelaye   
  750  jupyterlab                jupyterlab             stonebig   
  784  jupyterlab            jupyterlab-git           fcollonval   
  793  jupyterlab            jupyterlab-git       saulshanabrook   
  755  jupyterlab                jupyterlab            timkpaine   
  805  jupyterlab         jupyterlab_server            blink1073   
  627  jupyterlab                jupyterlab           IgorDouven   
  656  jupyterlab                jupyterlab            bionicles   
  734  jupyterlab                jupyterlab           nickeubank   
  789  jupyterlab            jupyterlab-git  mathematicalmichael   
  657  jupyterlab                jupyterlab              bjtho08   
  674  jupyterlab                jupyterlab         davidanthoff   
  683  jupyterlab                jupyterlab             echarles   
  690  jupyterlab                jupyterlab         flying-sheep   
  702  jupyterlab                jupyterlab               ivanov   
  703  jupyterlab                jupyterlab           jaipreet-s   
  705  jupyterlab                jupyterlab            jakirkham   
  768  jupyterlab                jupyterlab            yuvipanda   
  787  jupyterlab            jupyterlab-git           luispuerto   
  651  jupyterlab                jupyterlab            athornton   
  655  jupyterlab                jupyterlab              betatim   
  669  jupyterlab                jupyterlab           daejong123   
  673  jupyterlab                jupyterlab     david-waterworth   
  715  jupyterlab                jupyterlab              kevmk04   
  
      authorAssociation  author  
  706            MEMBER     419  
  658            MEMBER     232  
  708              NONE     146  
  700            MEMBER     126  
  762            MEMBER     124  
  745            MEMBER      47  
  752            MEMBER      47  
  684            MEMBER      44  
  643            MEMBER      43  
  622            MEMBER      21  
  785      COLLABORATOR      21  
  753            MEMBER      20  
  649            MEMBER      17  
  772            MEMBER      17  
  679

Chart({
  data:          org            repo             username authorAssociation  author
  860  nteract        hydrogen       wadethestealth            MEMBER     137
  840  nteract        hydrogen             aviatesk            MEMBER      66
  823  nteract       bookstore               mpacer            MEMBER      51
  822  nteract       bookstore              codecov              NONE      40
  832  nteract        hydrogen           BenRussert            MEMBER      36
  827  nteract       bookstore             willingc            MEMBER      35
  896  nteract       papermill                MSeal            MEMBER      31
  873  nteract         nteract         captainsafia            MEMBER      22
  906  nteract       papermill               rgbkrk            MEMBER      18
  886  nteract         nteract               rgbkrk            MEMBER      16
  826  nteract       bookstore                 todo              NONE      13
  889  nteract         nteract                stale              NONE      13
  890  nteract         nteract       wadethestealth            MEMBER      13
  909  nteract       scrapbook                MSeal            MEMBER      12
  865  nteract         nteract           BenRussert            MEMBER      11
  893  nteract      nteract.io                  now              NONE       9
  846  nteract        hydrogen           kiwi0fruit       CONTRIBUTOR       8
  901  nteract       papermill         duarteocarmo            MEMBER       7
  830  nteract        commuter               taniki       CONTRIBUTOR       6
  843  nteract        hydrogen             hoishing            MEMBER       6
  852  nteract        hydrogen            pepicello              NONE       6
  858  nteract        hydrogen             stevengj              NONE       6
  870  nteract         nteract         ShalokShalom              NONE       6
  874  nteract         nteract         davidanthoff              NONE       6
  905  nteract       papermill               mwouts            MEMBER       6
  821  nteract       bookstore                MSeal            MEMBER       5
  867  nteract         nteract       CrystallineCat            MEMBER       5
  883  nteract         nteract      nicolaskruchten              NONE       5
  908  nteract       papermill             willingc            MEMBER       5
  910  nteract       scrapbook              codecov              NONE       5
  917  nteract        semiotic               emeeks            MEMBER       5
  922  nteract            vdom             rmorshea            MEMBER       5
  828  nteract        commuter         captainsafia            MEMBER       4
  851  nteract        hydrogen         optionsraghu              NONE       4
  856  nteract        hydrogen             shr00mie              NONE       4
  857  nteract        hydrogen      sschuldenzucker              NONE       4
  891  nteract         nteract             willingc            MEMBER       4
  897  nteract       papermill          RoelBrouwer       CONTRIBUTOR       4
  898  nteract       papermill  avisrivastava254084              NONE       4
  911  nteract       scrapbook           matthiasdv            MEMBER       4
  824  nteract       bookstore               rgbkrk            MEMBER       3
  844  nteract        hydrogen             isajacas              NONE       3
  845  nteract        hydrogen          jesusloplar              NONE       3
  849  nteract        hydrogen             n-riesco            MEMBER       3
  864  nteract  minimal-plotly               rgbkrk            MEMBER       3
  894  nteract       papermill               0anton              NONE       3
  920  nteract            vdom              gnestor            MEMBER       3
  829  nteract        commuter               guzmud              NONE       2
  833  nteract        hydrogen               Helveg              NONE       2
  834  nteract        hydrogen            Mikeymice              NONE       2,
  encod

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [40]:
first_responders = created.copy().reset_index(drop=True)
for ii, issue in first_responders.iterrows():
    comments = issue.get('comments', [])['edges']
    if len(comments) == 0:
        continue
    i_comments = pd.DataFrame([ii['node'] for ii in comments]).sort_values('createdAt')
    first_comment = i_comments.iloc[0]
    first_responders.loc[ii, 'first_responder'] = first_comment['author']['login']
    first_responders.loc[ii, 'first_responder_association'] = first_comment['authorAssociation']

In [41]:
first_responders = (
    first_responders
    .groupby(['org', 'repo', 'first_responder', 'first_responder_association'])
    .count()
    .rename(columns={'author': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [42]:
n_plot = 50
for ii, (iorg, idata) in enumerate(first_responders.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='first_responder',
        y='count',
        color=alt.Color('first_responder_association', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    display(ch)

Chart({
  data:                org          repo first_responder first_responder_association  \
  0  binder-examples    demo-julia       choldgraf                      MEMBER   
  1  binder-examples  jupyter-rise       choldgraf                      MEMBER   
  
     count  
  0      1  
  1      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for binder-examples in the last 60 days',
  width: 1000
})

Chart({
  data:         org              repo first_responder first_responder_association  \
  4   ipython           ipython         Carreau                      MEMBER   
  2   ipython         ipykernel      ccordoba12                 CONTRIBUTOR   
  3   ipython           ipython    Benjamin-Lee                        NONE   
  5   ipython           ipython  LucianaMarques                 CONTRIBUTOR   
  6   ipython           ipython         dickmao                        NONE   
  7   ipython           ipython          dlukes                        NONE   
  8   ipython           ipython     drunkwcodes                        NONE   
  9   ipython           ipython       gdhameeja                        NONE   
  10  ipython           ipython        jfhbrook                        NONE   
  11  ipython  ipython-in-depth      jasongrout                      MEMBER   
  
      count  
  4      12  
  2       1  
  3       1  
  5       1  
  6       1  
  7       1  
  8       1  
  9       1  
  10      1  
  11      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for ipython in the last 60 days',
  width: 1000
})

Chart({
  data:         org                repo first_responder first_responder_association  \
  19  jupyter        jupyter-book       choldgraf                      MEMBER   
  65  jupyter         repo2docker         betatim                      MEMBER   
  42  jupyter            nbgrader        jhamrick                      MEMBER   
  34  jupyter           nbconvert           MSeal                      MEMBER   
  17  jupyter  enterprise_gateway     kevin-bates                      MEMBER   
  15  jupyter       docker-stacks         parente                      MEMBER   
  20  jupyter        jupyter-book         emdupre                COLLABORATOR   
  35  jupyter           nbconvert        t-makaro                COLLABORATOR   
  38  jupyter              nbdime         vidartf                COLLABORATOR   
  46  jupyter            nbviewer        krinsman                 CONTRIBUTOR   
  31  jupyter      jupyter_server        rolweber                COLLABORATOR   
  39  jupyter            nbgrader         Sefriol                        NONE   
  44  jupyter            nbgrader       perllaghu                        NONE   
  57  jupyter            notebook        jyu78749                        NONE   
  58  jupyter            notebook     kevin-bates                      MEMBER   
  68  jupyter         repo2docker          manics                        NONE   
  12  jupyter       accessibility          fperez                      MEMBER   
  13  jupyter       docker-stacks       daviddoji                        NONE   
  14  jupyter       docker-stacks   ejgutierrez74                        NONE   
  16  jupyter       docker-stacks        richtong                        NONE   
  18  jupyter             jupyter       choldgraf                      MEMBER   
  21  jupyter        jupyter-book     psychemedia                        NONE   
  22  jupyter        jupyter-book     thammegowda                 CONTRIBUTOR   
  23  jupyter      jupyter-sphinx    hugokerstens                 CONTRIBUTOR   
  24  jupyter      jupyter_client         dsblank                      MEMBER   
  25  jupyter      jupyter_client         jits023                        NONE   
  26  jupyter      jupyter_client     kevin-bates                      MEMBER   
  27  jupyter      jupyter_client         vidartf                        NONE   
  28  jupyter      jupyter_server   SylvainCorlay                      MEMBER   
  29  jupyter      jupyter_server       blink1073                      MEMBER   
  30  jupyter      jupyter_server        lresende                      MEMBER   
  32  jupyter      kernel_gateway     kevin-bates                      MEMBER   
  33  jupyter               nb2kg     kevin-bates                      MEMBER   
  36  jupyter              nbdime       bavaria95                        NONE   
  37  jupyter              nbdime      jaipreet-s                        NONE   
  40  jupyter            nbgrader       Yenthe666                        NONE   
  41  jupyter            nbgrader         dsblank                      MEMBER   
  43  jupyter            nbgrader         ocefpaf                        NONE   
  45  jupyter            nbgrader     psychemedia                        NONE   
  47  jupyter            notebook    Elizaveta239                        NONE   
  48  jupyter            notebook          Iorent                        NONE   
  49  jupyter            notebook   WilliamTjiong                        NONE   
  50  jupyter            notebook         YuGlede                        NONE   
  51  jupyter            notebook    autodataming                        NONE   
  52  jupyter            notebook        coldJune                        NONE   
  53  jupyter            notebook     dhirschfeld                 CONTRIBUTOR   
  54  jupyter            notebook  jamilraichouni                        NONE   
  55  jupyter            notebook      jasongrout                      MEMBER   
  56  jupyter    

Chart({
  data:                 org                repo first_responder  \
  76  jupyter-widgets          ipywidgets   SylvainCorlay   
  86  jupyter-widgets          ipywidgets         vidartf   
  89  jupyter-widgets           pythreejs         vidartf   
  77  jupyter-widgets          ipywidgets       beasteers   
  81  jupyter-widgets          ipywidgets      jasongrout   
  73  jupyter-widgets          ipyleaflet     dwr-psandhu   
  74  jupyter-widgets          ipyleaflet     paulathomas   
  75  jupyter-widgets          ipywidgets       JohnIssue   
  78  jupyter-widgets          ipywidgets          bybyte   
  79  jupyter-widgets          ipywidgets       choldgraf   
  80  jupyter-widgets          ipywidgets           hainm   
  82  jupyter-widgets          ipywidgets     jquacinella   
  83  jupyter-widgets          ipywidgets         kafonek   
  84  jupyter-widgets          ipywidgets          piyueh   
  85  jupyter-widgets          ipywidgets      rskabelund   
  87  jupyter-widgets          ipywidgets     zxzhaixiang   
  88  jupyter-widgets  jupyterlab-sidecar      doutriaux1   
  90  jupyter-widgets            tutorial         vidartf   
  
     first_responder_association  count  
  76                      MEMBER      5  
  86                      MEMBER      4  
  89                      MEMBER      4  
  77                        NONE      2  
  81                      MEMBER      2  
  73                        NONE      1  
  74                        NONE      1  
  75                        NONE      1  
  78                        NONE      1  
  79                        NONE      1  
  80                        NONE      1  
  82                        NONE      1  
  83                        NONE      1  
  84                        NONE      1  
  85                        NONE      1  
  87                        NONE      1  
  88                        NONE      1  
  90                      MEMBER      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for jupyter-widgets in the last 60 days',
  width: 1000
})

Chart({
  data:             org                     repo     first_responder  \
  95   jupyterhub                binderhub             betatim   
  121  jupyterhub      mybinder.org-deploy             betatim   
  114  jupyterhub               jupyterhub             ryogesh   
  132  jupyterhub             team-compass             betatim   
  143  jupyterhub   zero-to-jupyterhub-k8s        consideRatio   
  125  jupyterhub              nbgitpuller           yuvipanda   
  135  jupyterhub  the-littlest-jupyterhub      GeorgianaElena   
  139  jupyterhub  the-littlest-jupyterhub           yuvipanda   
  133  jupyterhub             team-compass           choldgraf   
  134  jupyterhub             team-compass               minrk   
  142  jupyterhub              wrapspawner             rkdarst   
  93   jupyterhub             batchspawner             rkdarst   
  98   jupyterhub                binderhub           sgibson91   
  108  jupyterhub               jupyterhub            katsar0v   
  113  jupyterhub               jupyterhub          ryanlovett   
  118  jupyterhub              kubespawner        consideRatio   
  137  jupyterhub  the-littlest-jupyterhub     lachlancampbell   
  145  jupyterhub   zero-to-jupyterhub-k8s              manics   
  146  jupyterhub   zero-to-jupyterhub-k8s   sampathkethineedi   
  91   jupyterhub             batchspawner               Hoeze   
  92   jupyterhub             batchspawner            rcthomas   
  94   jupyterhub                binderhub               arnim   
  96   jupyterhub                binderhub        consideRatio   
  97   jupyterhub                binderhub              manics   
  99   jupyterhub            dockerspawner            Redhappy   
  100  jupyterhub            dockerspawner    ZelphirKaltstahl   
  101  jupyterhub            dockerspawner           timhughes   
  102  jupyterhub            dockerspawner         yakutovicha   
  103  jupyterhub   jupyter-rsession-proxy              xhochy   
  104  jupyterhub     jupyter-server-proxy     juliusvonkohout   
  105  jupyterhub               jupyterhub      GeorgianaElena   
  106  jupyterhub               jupyterhub         UCSBGauchos   
  107  jupyterhub               jupyterhub           casperdcl   
  109  jupyterhub               jupyterhub            krinsman   
  110  jupyterhub               jupyterhub              manics   
  111  jupyterhub               jupyterhub               minrk   
  112  jupyterhub               jupyterhub          rochaporto   
  115  jupyterhub               jupyterhub           vilhelmen   
  116  jupyterhub               jupyterhub            wqh17101   
  117  jupyterhub    kerberosauthenticator              jcrist   
  119  jupyterhub              kubespawner              gsemet   
  120  jupyterhub              kubespawner      julienchastang   
  122  jupyterhub      mybinder.org-deploy           choldgraf   
  123  jupyterhub              nbgitpuller      albertmichaelj   
  124  jupyterhub              nbgitpuller            mpilosov   
  126  jupyterhub           oauthenticator  hedgehog-millenium   
  127  jupyterhub           oauthenticator            inviscid   
  128  jupyterhub           oauthenticator               minrk   
  129  jupyterhub           oauthenticator           yuvipanda   
  130  jupyterhub           systemdspawner            rstuckey   
  
      first_responder_association  count  
  95                       MEMBER      9  
  121                      MEMBER      8  
  114                 CONTRIBUTOR      7  
  132                      MEMBER      6  
  143                      MEMBER      5  
  125                      MEMBER      4  
  135                COLLABORATOR      4  
  139                      MEMBER      4  
  133                      MEMBER      3  
  134                      MEMBER      3  
  142                        NONE      3  
  93                  CONTRIBUTOR      2  
  98                       MEMBER      2  
  108          

Chart({
  data:             org                       repo  first_responder  \
  174  jupyterlab                 jupyterlab       jasongrout   
  162  jupyterlab                 jupyterlab        blink1073   
  190  jupyterlab                 jupyterlab          vidartf   
  178  jupyterlab                 jupyterlab       krassowski   
  194  jupyterlab   jupyterlab-data-explorer   saulshanabrook   
  171  jupyterlab                 jupyterlab       ian-r-rose   
  152  jupyterlab                 jupyterlab           BoPeng   
  153  jupyterlab                 jupyterlab       IgorDouven   
  159  jupyterlab                 jupyterlab           afshin   
  166  jupyterlab                 jupyterlab         echarles   
  167  jupyterlab                 jupyterlab        ellisonbg   
  175  jupyterlab                 jupyterlab            jtpio   
  185  jupyterlab                 jupyterlab       telamonian   
  188  jupyterlab                 jupyterlab          tslaton   
  149  jupyterlab  extension-cookiecutter-ts       jasongrout   
  150  jupyterlab          jupyter-renderers        jonmmease   
  151  jupyterlab                 jupyterlab      AlokTakshak   
  154  jupyterlab                 jupyterlab        Koushik95   
  155  jupyterlab                 jupyterlab         SamKacer   
  156  jupyterlab                 jupyterlab         SgtPooki   
  157  jupyterlab                 jupyterlab  SultanOrazbayev   
  158  jupyterlab                 jupyterlab            Zlass   
  160  jupyterlab                 jupyterlab       alesgenova   
  161  jupyterlab                 jupyterlab         aschlaep   
  163  jupyterlab                 jupyterlab      claraabello   
  164  jupyterlab                 jupyterlab       daejong123   
  165  jupyterlab                 jupyterlab       doutriaux1   
  168  jupyterlab                 jupyterlab     felixdollack   
  169  jupyterlab                 jupyterlab     flying-sheep   
  170  jupyterlab                 jupyterlab         groverj3   
  172  jupyterlab                 jupyterlab          ibdafna   
  173  jupyterlab                 jupyterlab       jaipreet-s   
  176  jupyterlab                 jupyterlab        kanekalla   
  177  jupyterlab                 jupyterlab        kapilkd13   
  179  jupyterlab                 jupyterlab    markellekelly   
  180  jupyterlab                 jupyterlab        marwahaha   
  181  jupyterlab                 jupyterlab     michael-burt   
  182  jupyterlab                 jupyterlab   mouradmourafiq   
  183  jupyterlab                 jupyterlab   saulshanabrook   
  184  jupyterlab                 jupyterlab         stonebig   
  186  jupyterlab                 jupyterlab        tgeorgeux   
  187  jupyterlab                 jupyterlab        timkpaine   
  189  jupyterlab                 jupyterlab            van-8   
  191  jupyterlab        jupyterlab-celltags            e01n0   
  192  jupyterlab      jupyterlab-commenting          gogasca   
  193  jupyterlab      jupyterlab-commenting           hoo761   
  195  jupyterlab             jupyterlab-git      DiegoPiloni   
  196  jupyterlab             jupyterlab-git       fcollonval   
  197  jupyterlab             jupyterlab-git       jaipreet-s   
  198  jupyterlab    jupyterlab-google-drive       ian-r-rose   
  
      first_responder_association  count  
  174                      MEMBER     25  
  162                      MEMBER      9  
  190                      MEMBER      9  
  178                      MEMBER      4  
  194                      MEMBER      4  
  171                      MEMBER      3  
  152                      MEMBER      2  
  153                        NONE      2  
  159                      MEMBER      2  
  166                      MEMBER      2  
  167                      MEMBER      2  
  175                      MEMBER      2  
  185                      MEMBER      2  
  188                 CONTRIBUTOR      2  
  149                  

Chart({
  data:          org            repo  first_responder first_responder_association  \
  222  nteract       papermill            MSeal                      MEMBER   
  215  nteract         nteract     captainsafia                      MEMBER   
  211  nteract        hydrogen   wadethestealth                      MEMBER   
  204  nteract       bookstore           mpacer                      MEMBER   
  219  nteract         nteract   wadethestealth                      MEMBER   
  224  nteract       scrapbook            MSeal                      MEMBER   
  206  nteract       bookstore         willingc                      MEMBER   
  209  nteract        hydrogen         aviatesk                      MEMBER   
  203  nteract       bookstore            MSeal                      MEMBER   
  205  nteract       bookstore           rgbkrk                      MEMBER   
  207  nteract              dx       dylanross3                      MEMBER   
  208  nteract        hydrogen       BenRussert                      MEMBER   
  210  nteract        hydrogen         n-riesco                      MEMBER   
  212  nteract  minimal-plotly  nicolaskruchten                        NONE   
  213  nteract         nteract       BenRussert                      MEMBER   
  214  nteract         nteract          bnbarak                      MEMBER   
  216  nteract         nteract             drbh                        NONE   
  217  nteract         nteract      hydrosquall                      MEMBER   
  218  nteract         nteract           rgbkrk                      MEMBER   
  220  nteract         nteract         willingc                      MEMBER   
  221  nteract         nteract       yogevyuval                        NONE   
  223  nteract       papermill         willingc                      MEMBER   
  225  nteract        semiotic      micahstubbs                 CONTRIBUTOR   
  226  nteract            vdom         rmorshea                      MEMBER   
  
       count  
  222      8  
  215      5  
  211      4  
  204      3  
  219      3  
  224      3  
  206      2  
  209      2  
  203      1  
  205      1  
  207      1  
  208      1  
  210      1  
  212      1  
  213      1  
  214      1  
  216      1  
  217      1  
  218      1  
  220      1  
  221      1  
  223      1  
  225      1  
  226      1  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'first_responder_association',
      scale: Scale({
        domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
        range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
      }),
      type: 'nominal'
    }),
    x: X({
      field: 'first_responder',
      type: 'nominal'
    }),
    y: Y({
      field: 'count',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Top 50 first responders for nteract in the last 60 days',
  width: 1000
})

In [43]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>